# Building Agents with Semantic Kernel

This notebook demonstrates how to create and orchestrate agents workflows using Semantic Kernel.

**Objectives:**
- Understand how to define and use agents with Semantic Kernel.
- Learn to orchestrate agent group chats and termination strategies.

## Setup

In this section, we will set up the Semantic Kernel environment and configure it to use different LLM providers.

**Step 1**: Install NuGet packages

To get started with Semantic Kernel, you need to install the required NuGet packages. These packages provide the core functionality for interacting with AI models and managing environment variables. Specifically:
- `Microsoft.SemanticKernel` enables you to build and run AI-powered workflows.
- `DotNetEnv` allows you to load environment variables from a `.env` file, making it easier to manage secrets and configuration settings.

In [1]:
// Import Semantic Kernel and required packages
#r "nuget: Microsoft.SemanticKernel, 1.55.0"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7 Microsoft.SemanticKernel, 1.55.0 Microsoft.SemanticKernel.Agents.Core, 1.55.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.

In [2]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from e:\profile_dev\aiagent-workshop\notebooks\semantic-kernel\../..\.env


**Step 3**: Instantiate the Kernel and Chat Clients

The Semantic Kernel is the core component that orchestrates AI services and plugins. In this step, we create and configure a Kernel instance, which will be used to interact with AI models and agents.

In [44]:
using System.ClientModel;
using OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Text;

OpenAIClient client = null;
if(Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true")
{
    // Configure Azure OpenAI client
    var azureEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(azureEndpoint) });
}
else if(Environment.GetEnvironmentVariable("USE_OPENAI") == "true")
{
    // Configure OpenAI client
    var apiKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey));
}
else if(Environment.GetEnvironmentVariable("USE_GITHUB") == "true")
{
    // Configure GitHub model client
    var uri = Environment.GetEnvironmentVariable("GITHUB_MODEL_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("GITHUB_TOKEN");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(uri) });
}

var modelId = Environment.GetEnvironmentVariable("MODEL");
// Create a chat completion service
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(modelId, client); 

# Weather Agent

This section demonstrates how to define and use a weather agent that calls an external weather API using a plugin. The agent will respond to user queries about the weather in a given city.

**Step 1:**: Create the plugin to call the weather API.
Replace `YOUR_OPENWEATHERMAP_API_KEY` with your actual API key to enable real API calls.

In [45]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI; 
using System.Net.Http;
using System.Text.Json;

public class WeatherPlugin
{
    private readonly HttpClient _httpClient = new HttpClient();

    [KernelFunction]
    public async Task<string> GetRandomWeather(string city)
    {
        var httpClient = new HttpClient();
        var apiKey = Environment.GetEnvironmentVariable("OPENWEATHERMAP_API_KEY");
        var url = $"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={apiKey}&units=metric";
        var response = await httpClient.GetAsync(url);
        response.EnsureSuccessStatusCode();
        var json = await response.Content.ReadAsStringAsync();
        return json;
    }
}

**Step 2**: Create the weather agent and register it with the Kernel

In [46]:

var weatherAgentName = "WeatherAgent";
var weatherAgentInstructions = @"
You are a weather assistant. Use the WeatherPlugin to answer questions about the weather in any city.
If the user asks for other information, politely decline and suggest they ask about the weather instead.
If the city is not specified, ask the user to provide a city name.
If the weather data is not available, inform the user that the weather information could not be retrieved.
";

// Get the chat completion service
Kernel kernel = builder.Build();
// Register the plugin with the kernel
var weatherPlugin = new WeatherPlugin();
kernel.Plugins.AddFromObject(weatherPlugin, "WeatherPlugin");

// Define the weather agent
ChatCompletionAgent weatherAgent = new()
{
    Name = weatherAgentName,
    Instructions = weatherAgentInstructions,
    Kernel = kernel,
};


**Step 3**: Invoke the agent with a specific city

In [47]:
var invokeAgentAsync = async (string input) =>
{
    await foreach (var message in weatherAgent.InvokeAsync(input))
    {
        Console.WriteLine(message.Message.Content);
    }
};

await invokeAgentAsync("What is the weather in Australia?");

Australia is a large country with diverse weather depending on the region. Could you please specify a city or region in Australia so I can provide detailed weather information?


## Step 2: Multi-Agent Chat Example (Drafter & Reviewer)

This section demonstrates a multi-agent chat with a Drafter Agent and a Reviewer Agent. The Drafter creates a draft response for a given topic, and the Reviewer reviews and approves or suggests changes. The conversation is limited to two iterations.

In [97]:
// Define agent instructions
string writerAgentName = "WriterAgent";
string writerInstructions = @"
You are WriterAgent, a helpful and knowledgeable assistant. Your task is to draft a concise, clear, and informative response on the given topic. 
- Focus on accuracy, clarity, and relevance.
- Use professional language and structure your response logically.
- Avoid unnecessary jargon or overly complex explanations.
- Limit your draft to 3-5 sentences.
- After drafting, wait for the ReviewerAgent's feedback before making any changes.
";

// Create Drafter Agent
ChatCompletionAgent writerAgent = new()
{
    Name = writerAgentName,
    Instructions = writerInstructions,
    Kernel = kernel,
};


In [104]:

string reviewerAgentName = "ReviewerAgent";
string reviewerInstructions = @"
You are ReviewerAgent, a critical and highly demanding reviewer. Your job is to rigorously review the WriterAgent's draft and either approve it or provide detailed, specific improvements.
- Only reply with 'Approved.' if the draft is exceptionally clear, accurate, and perfectly structured.
- Be direct, concise, and do not tolerate vague, incomplete, or imprecise answers.
- Do not engage in chit chat or pleasantries.
- Always prioritize thoroughness and precision in your feedback.
";

// Create Reviewer Agent
ChatCompletionAgent reviewerAgent = new()
{
    Name =reviewerAgentName,
    Instructions = reviewerInstructions,
    Kernel = kernel,
};

In [105]:
using Microsoft.SemanticKernel.Agents.Chat;    
    
#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.
KernelFunction selectionFunction =
    AgentGroupChat.CreatePromptFunctionForStrategy(
        $$$"""
        Based on the conversation history, determine which participant should take the next turn.
        Only state the name of the participant to take the next turn.
        No participant should take more than one turn in a row.

        Participants:
        - {{{writerAgentName}}}
        - {{{reviewerAgentName}}}

        Rules:
        - After user input, it is {{{writerAgentName}}}'s turn.
        - After {{{writerAgentName}}}, it is {{{reviewerAgentName}}}'s turn.
        - After {{{reviewerAgentName}}}, it is {{{writerAgentName}}}'s turn.

        Conversation history:
        {{$history}}
        """,
        safeParameterNames: "history");

KernelFunction terminationFunction =
    AgentGroupChat.CreatePromptFunctionForStrategy(
        $$$"""
        Review the conversation history to determine if the review process is complete.
        If the most recent message from {{{reviewerAgentName}}} contains 'Approved.', respond with 'yes'.
        Otherwise, respond with 'no'.

        Conversation history:
        {{$history}}
        """,
        safeParameterNames: "history");

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
var executionSettings = new AgentGroupChatSettings()
{
    SelectionStrategy =
new KernelFunctionSelectionStrategy(selectionFunction, kernel)
{
    HistoryVariableName = "history",
    HistoryReducer = new ChatHistoryTruncationReducer(1),
    ResultParser = (result) => result.GetValue<string>() ?? writerAgent.Name!,
},
    TerminationStrategy =
new KernelFunctionTerminationStrategy(terminationFunction, kernel)
{
    HistoryVariableName = "history",
    MaximumIterations = 6,
    //HistoryReducer = new ChatHistoryTruncationReducer(2),
    ResultParser = (result) => result.GetValue<string>()?.Contains("yes", StringComparison.OrdinalIgnoreCase) ?? false,
}};


In [106]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
// Create the group chat
AgentGroupChat groupChat = new(writerAgent, reviewerAgent)
{
    ExecutionSettings = executionSettings,
};
#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

// Take a topic as input
string topic = "What is the future of AI in healthcare?";
Console.WriteLine($"User Topic: {topic}\n");
groupChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, topic));

// Run the group chat and print the conversation
int turn = 1;
await foreach (var message in groupChat.InvokeAsync())
{
    Console.WriteLine($"Turn {turn++} [{message.Role}]: {message.Content}\n");
}
Console.WriteLine($"[Chat Complete: {groupChat.IsComplete}]");





User Topic: What is the future of AI in healthcare?



Error: Command cancelled.